## Set parameters for queries

In [1]:
import ipywidgets as widgets
from ipywidgets import HBox

# all available parameters
tasks = [('Cutting Action',"cut:CuttingAction"),
        ('Quartering', "cut:Quartering"),
        ('Julienning',"cut:Julienning"),
        ('Halving',"cut:Halving"),
        ('Dicing',"soma:Dicing"),
        ('Cutting',"soma:Cutting"),
        ('Slicing',"soma:Slicing"),
        ('Snipping',"cut:Snipping"),
        ('Slivering',"cut:Slivering"),
        ('Sawing',"cut:Sawing"),
        ('Paring',"cut:Paring"),
        ('Carving',"cut:Carving"),
        ('Mincing',"cut:Mincing"),
        ('Cubing',"cut:Cubing"),
        ('Chopping',"cut:Chopping")]

objects=[('almond', "obo:FOODON_00003523"),
      ('cucumber', "obo:FOODON_00003415"),
      ('strawberry', "obo:FOODON_00003443"),
      ('coconut', "obo:FOODON_00003449"),
      ('pineapple', "obo:FOODON_00003459"),
      ('pumpkin', "obo:FOODON_00003486"),
      ('pepper', "obo:FOODON_00003520"),
      ('squash', "obo:FOODON_00003539"),
      ('lime', "obo:FOODON_00003661"),
      ('banana', "obo:FOODON_00004183"),
      ('cherry', "obo:FOODON_03301240"),
      ('bean', "obo:FOODON_03301403"),
      ('lemon', "obo:FOODON_03301441"),
      ('apple', "obo:FOODON_03301710"),
      ('citron', "obo:FOODON_03306596"),
      ('kumquat', "obo:FOODON_03306597"),
      ('orange', "obo:FOODON_03309832"),
      ('tomato', "obo:FOODON_03309927"),
      ('olive', "obo:FOODON_03317509")]

task=""
tobject=""

# Takes a Task of the tasks list. 
# Then makes the previously initialized task variable global and assigns value of the given Task to it 
def chooseTask(Task):
    global task
    task=Task
    
# Takes an Object of the object list.
# Then makes the previously initialized tobject variable global and assigns value of the given Object to it 
def chooseObject(Object):
    global tobject
    tobject=Object
    
# Create the dropdown widgets
task_widget = widgets.Dropdown(options=tasks, description='Task:')
object_widget = widgets.Dropdown(options=objects, description='Object:')

# Define the event handlers
def taskEvent(event):
    chooseTask(event.new)

def objectEvent(event):
    chooseObject(event.new)
    
# Attach the event handlers to the widgets
task_widget.observe(taskEvent, names='value')
object_widget.observe(objectEvent, names='value')

# Combine the widgets using HBox
widgets_display = HBox([task_widget, object_widget])

# Display widgets
widgets_display

# optional: set Parameters manually:
#task = "cut:Quartering"
#tobject = "obo:FOODON_03301710"

## import rdflib for querying the knowledge graph for action parameters

In [2]:
!pip install rdflib
# import rdflib
from rdflib import Graph, Literal, URIRef, Namespace
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc
from rdflib.namespace import OWL, RDF, RDFS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 7.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.3 MB/s eta 0:00:00


# Now let us start with querying the knowledge graph!
## We can query for the tool to be used for the chosen task:

In [3]:
g = Graph()

# define prefixes to be used in the query 
SOMA = Namespace("http://www.ease-crc.org/ont/SOMA.owl#")
CUT2 = Namespace("http://www.ease-crc.org/ont/situation_awareness#")
CUT = Namespace("http://www.ease-crc.org/ont/food_cutting#")
DUL = Namespace("http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#")
OBO = Namespace("http://purl.obolibrary.org/obo/")
g.bind("owl", OWL)
g.bind("rdfs", RDFS)
g.bind("soma", SOMA)
g.bind("cut2", CUT2)
g.bind("cut", CUT)
g.bind("dul", DUL)
g.bind("obo", OBO)

# the actual query comes here
query = """SELECT ?alltools WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {
      %s rdfs:subClassOf ?node.
      ?node owl:onProperty soma:hasDisposition.
      ?node owl:someValuesFrom ?collection.
      ?collection owl:intersectionOf ?node2.
      ?node2 rdf:first cut2:Cuttability.
      ?node2 rdf:rest ?toolnode.
      ?toolnode rdf:rest ?collection2.
      ?collection2 rdf:first ?tooluse.
      ?tooluse owl:onProperty soma:affordsTrigger.
      ?tooluse owl:allValuesFrom ?tool.
      ?tool owl:onProperty dul:classifies.
      ?tool owl:allValuesFrom ?whattool.
      ?alltools rdfs:subClassOf* ?whattool.} }""" % (tobject) # the input parameter for the query. For testing, you can remove % (tobject) here
#and replace %s in the first line of the query with a variable like ?food to get all food and objects

for row in g.query(query):
    print(row.alltools)

http://www.ease-crc.org/ont/food_cutting#Knife
http://www.ease-crc.org/ont/food_cutting#ParingKnife
http://www.ease-crc.org/ont/situation_awareness#BreadKnife
http://www.ease-crc.org/ont/SOMA-HOME.owl#KitchenKnife


# The robot needs to know if prior actions need to be executed:

In [5]:
additionalactionquery = """SELECT ?action WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  %s rdfs:subClassOf ?restriction.
  ?restriction owl:onProperty cut:hasPart.
  ?restriction owl:someValuesFrom ?node.
  ?node owl:intersectionOf ?intersec.
  ?intersec rdf:first ?foodpart.
  ?intersec rdf:rest ?node2.
  ?node2 rdf:first ?part.
  ?part owl:onProperty cut:hasEdibility.
  {?part owl:someValuesFrom cut:ShouldBeAvoided.}
  UNION
  {?part owl:someValuesFrom cut:MustBeAvoided.}
  ?foodpart rdfs:subClassOf ?partrestriction.
  ?partrestriction owl:onProperty soma:hasDisposition.
  ?partrestriction owl:someValuesFrom ?partnode.
  ?partnode owl:intersectionOf ?intersection.
  ?intersection rdf:first ?disposition.
  ?intersection rdf:rest ?actionrestriction.
  ?actionrestriction rdf:first ?actionnode.
  ?actionnode owl:onProperty soma:affordsTask.
  ?actionnode owl:someValuesFrom ?action.
} }""" % (tobject) # the input parameter for the query. For testing, you can remove % (tobject) here
#and replace %s in the first line of the query with a variable like ?food to get all food and objects

for row in g.query(additionalactionquery):
    print(row.action)

http://www.ease-crc.org/ont/food_cutting#CoreScraping
http://www.ease-crc.org/ont/food_cutting#CoreCutting


In [10]:
prioractionquery = """  SELECT ?priortask WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  %s rdfs:subClassOf ?sub.
  ?sub owl:onProperty cut:requiresPriorTask .
  ?sub owl:someValuesFrom ?priortask.
} }""" % (task)

for row in g.query(prioractionquery):
    print(row.priortask)

http://www.ease-crc.org/ont/food_cutting#Halving


# The robot needs to also know the number of repetitions required for performing the task:

In [29]:
repetitionsquery = """  SELECT ?rep WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  {
     OPTIONAL{ %s rdfs:subClassOf ?action}
        ?action rdfs:subClassOf* ?rep_node.
        ?rep_node owl:onProperty cut:repetitions.
        FILTER EXISTS {
            ?rep_node owl:hasValue ?val.}
        BIND("= 1" AS ?rep)}
    UNION
    {
       OPTIONAL{ %s rdfs:subClassOf ?action }
        ?action rdfs:subClassOf* ?rep_node.
        ?rep_node owl:onProperty cut:repetitions.
        FILTER EXISTS {
            ?rep_node owl:minQualifiedCardinality ?val.}
        BIND("more than 1" AS ?rep)}} }""" % (task, task)

for row in g.query(repetitionsquery):
    print(row.rep)

= 1


# Last thin to know before performing the task: The position needed for action execution:

In [19]:
positionquery = """  SELECT ?position WHERE {
      SERVICE <https://api.krr.triply.cc/datasets/mkumpel/FruitCuttingKG/services/FoodCuttingKG/sparql> {  
  OPTIONAL { %s rdfs:subClassOf ?sub.}
  ?sub rdfs:subClassOf* ?node.
  ?node owl:onProperty cut:affordsPosition.
  ?node owl:someValuesFrom ?position.
} }""" % (task)

for row in g.query(positionquery):
    print(row.position)

http://www.ease-crc.org/ont/food_cutting#halving_position
